In [ ]:
import pandas as pd
import glob
import pathlib
import datetime

import unidecode
import re
from sodapy import Socrata
from datetime import datetime
import numpy as np

***
# Información complementaria a los análisis

Como información complementaria, se extrae de la página [datos.gov.co](https://www.datos.gov.co/) la Base de Datos Única de Afiliados (BDUA) de los regímenes subsidiado y contributivo. Esta información contribuye con datos clave sobre la población afiliada al Sistema General de Seguridad Social en Salud, permitiendo analizar la cobertura, características demográficas y distribución de los afiliados.

Para la descarga y procesamiento de los datos se utilizaron las librerías de Python, específicamente `from sodapy import Socrata`, empleando los códigos de las tablas `d7a5-cnra` (subsidiado) y `tq4m-hmg2` (contributivo). El siguiente fragmento ilustra el proceso técnico de extracción y consolidación de la información:


In [ ]:
DSID = [
        #'d7a5-cnra',   # Población Base de Datos Única de Afiliados BDUA del régimen subsidiado  (1030170, 14) tiempo = 5 minutos
        'tq4m-hmg2'    # Población Base de Datos Única de Afiliados BDUA del régimen contributivo (466421, 14) tiempo = 3 minutos
       ]

DFCONSOL = []
for j in DSID:
    client = Socrata("www.datos.gov.co", None)
    results = client.get_all(j)
    results_df = pd.DataFrame.from_records(results)
    DFCONSOL += [results_df]
    metadata = client.get_metadata(j)
    NREG = metadata['downloadCount']
    FECHAA = datetime.fromtimestamp(metadata['rowsUpdatedAt']).strftime('%Y-%m-%d')
    print(j, "->",
          NREG, "->",
          FECHAA, "->",
          results_df.shape)


tq4m-hmg2 -> 4870 -> 2025-05-08 -> (466421, 14)


In [ ]:
#pd.concat(DFCONSOL).sample(100, random_state=1).to_excel("analisis exploratorio.xlsx")

***
# 03_Ampliacion del analisis para Chocó, Cauca y Córdoba en el régimen contributivo



In [27]:
contri = pd.concat(DFCONSOL)
SELECT_DEP = {'CHOCO' : "27",
              'CORDOBA' : "23",
              'CAUCA' : "19"}
DF_SELECT_DEP = contri[contri['dpr_nombre'].isin(SELECT_DEP.keys())]  # Se filtra por los departamentos mas afectados
DF_SELECT_DEP.head(2)

,tps_gnr_nombre,tps_grp_etr_id,ent_id,ent_nombre,tps_rgm_nombre,tps_afl_nombre,tps_est_afl_nombre,tps_cnd_bnf_nombre,zns_nombre,dpr_nombre,mnc_nombre,tps_grp_pbl_id,cantidad,tps_nvl_ssb_id
11,Masculino,5 a 15,EPS002,SALUD TOTAL ENTIDAD PROMOTORA DE SALUD DEL REG...,Contributivo,BENEFICIARIO,Activo,NO APLICA,Urbana-Cabecera Municipal,CORDOBA,CERETE,NULL,31,NaN
13,Femenino,1 a 5,EPS041,NUEVA EPS S.A. -CM,Contributivo,BENEFICIARIO,Activo,NO APLICA,Urbana,CAUCA,CALDONO,COMUNIDADES INDÍGENAS,1,N


In [59]:
set(DF_SELECT_DEP ['tps_grp_etr_id'])


{'1 a 5',
 '15 a 19',
 '19 a 45',
 '45 a 50',
 '5 a 15',
 '50 a 55',
 '55 a 60',
 '60 a 65',
 '65 a 70',
 '70 a 75',
 '< 1',
 '> 75'}

In [53]:
DF_SELECT_DEP.columns

Index(['tps_gnr_nombre', 'tps_grp_etr_id', 'ent_id', 'ent_nombre',
       'tps_rgm_nombre', 'tps_afl_nombre', 'tps_est_afl_nombre',
       'tps_cnd_bnf_nombre', 'zns_nombre', 'dpr_nombre', 'mnc_nombre',
       'tps_grp_pbl_id', 'cantidad', 'tps_nvl_ssb_id', 'DP'],
      dtype='object')

In [68]:
dfpivot = DF_SELECT_DEP.pivot_table(index = ['DP', 'dpr_nombre'],
                          columns = 'tps_gnr_nombre',
                          values='mnc_nombre',
                            aggfunc="count").reset_index()
dfpivot


tps_gnr_nombre,DP,dpr_nombre,Femenino,Masculino
0,19,CAUCA,8085,8451
1,23,CORDOBA,9487,10080
2,27,CHOCO,2469,2634


In [ ]:
import folium
import json
import pandas as pd

ruta_geojson = "02_Tablas_Normalizadas\\colombia.geo.json"

# Cargar datos geojson
with open(ruta_geojson, 'r', encoding='utf-8') as f:
    municipios_geojson = json.load(f)

# Crear DataFrame con información
info_df = pd.DataFrame({
    'DP': ['19', '23', '27'],
    'Departamento': ['CAUCA', 'CORDOBA', 'CHOCO'],
    'Femenino': [8085, 9487, 2469],
    'Masculino': [8451, 10080, 2634]
})

# Filtrar municipios presentes en el DataFrame
municipios_filtrados = {
    "type": "FeatureCollection",
    "features": [f for f in municipios_geojson['features'] if f['properties']['DPTO'] in info_df['DP'].tolist()]
}

# Crear mapa centrado en Colombia
m = folium.Map(location=[5, -75], zoom_start=6)

# Añadir municipios con información del DataFrame en tooltips y popups
for feature in municipios_filtrados['features']:
    dp = feature['properties']['DPTO']
    dept_info = info_df[info_df['DP'] == dp].iloc[0]
    folium.GeoJson(
        feature,
        style_function=lambda x: {
            'fillColor': '#F03B20',
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.7
        },
        highlight_function=lambda x: {
            'fillColor': '#ffff00',
            'color': 'black',
            'weight': 2,
            'fillOpacity': 0.9
        },
        tooltip=folium.GeoJsonTooltip(
            fields=['NOMBRE_DPT'],
            aliases=['Departamento:'],
            localize=True,
            labels=True,
            sticky=False,
            style="background-color: #F0EFEF; border: 2px solid black; border-radius: 3px;",
            max_width=300
        ),
        popup=folium.Popup(
            f"<b>Departamento:</b> {dept_info['Departamento']}<br>"
            f"<b>Femenino:</b> {dept_info['Femenino']}<br>"
            f"<b>Masculino:</b> {dept_info['Masculino']}",
            max_width=250
        )
    ).add_to(m)

m.save("mapa_municipios_con_datos.html")
print("Mapa guardado como mapa_municipios_con_datos.html")


Mapa guardado como mapa_municipios_con_datos.html
